In [1]:
import requests
from bs4 import BeautifulSoup


from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait

import re
import time

import csv

from tqdm import tqdm

import json

import pandas as pd

# 실전용 

In [2]:
# 창 띄우기
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [ ]:
# categories = {
#     '상의' : ['니트', '후드', '맨투맨','와이셔츠','블라우스','긴소매 티셔츠','반소매 티셔츠','민소매 티셔츠','카라 티셔츠','베스트'],
#     '바지' : ['데님 팬츠', '슬랙스','트레이닝바지','조거팬츠','숏팬츠','코튼 팬츠','레깅스', '와이드 팬츠'],
#     '아우터' : ['후드 집업', '바람막이', '코트', '롱패딩', '숏패딩', '패딩 베스트', '블루종/MA-1', 
#                 '레더 자켓', '무스탕', '트러커 자켓', '블레이저', '가디건','뽀글이 후리스','사파리 자켓'],
#     '원피스' : ['미니 원피스', '미디 원피스', '투피스', '롱원피스', '점프수트'],
#     '스커트' : ['미니 스커트', '미디 스커트', '롱 스커트'],
# }

In [3]:
categories = {
    '상의' : ['카라 티셔츠','베스트'],
    '바지' : ['데님 팬츠', '슬랙스','트레이닝바지','조거팬츠','숏팬츠','코튼 팬츠','레깅스', '와이드 팬츠'],
    '아우터' : ['후드 집업', '바람막이', '코트', '롱패딩', '숏패딩', '패딩 베스트', '블루종', 
                '레더 자켓', '무스탕', '트러커 자켓', '블레이저', '가디건','뽀글이 후리스','사파리 자켓'],
    '원피스' : ['미니 원피스', '미디 원피스', '투피스', '롱원피스', '점프수트'],
    '스커트' : ['미니 스커트', '미디 스커트', '롱 스커트']
}


# 완료

# 상의 : [후드, 맨투맨, 와이셔츠, 블라우스, 긴소매티셔츠, 민소매 티셔츠]


- 검색 화면 진입 후 결과 추출하기

- 필요 정보 들 추출하기

- 가장 큰 박스 : div[class="sc-2e746fd3-0 loRRgN"]

    - 작은 박스 1 : div[class="sc-2e746fd3-1 gHhzqc"] -> connect_url, 이미지_url
        - connect_url : a[href]
            - 이미지_url : connect_url의 a[href] > div > img[class="sc-b34a2ec2-0 bOxxOH"] 
    
    
    - 작은 박스 2 : div[class="sc-2e746fd3-6 cVhNfy"] -> 가격, 제목

        - 가격 : div[class="sc-2e746fd3-5 laAErS"]
        - 제목 : div[class="sc-2e746fd3-5 RmlFc"]

In [15]:
for i, (main, subs) in enumerate(categories.items()) :
    for sub in subs :
        time.sleep(2)
        
        connect_urls = []
        prices = []
        img_urls = []
        titles = []
        sizes = []


        # secondwear 사이트 접속하기
        driver.get('https://www.hellomarket.com/')

        # 검색창 들어가기
        pre_search_part = driver.find_element(By.CSS_SELECTOR, 'div[class="sc-90f95001-3 eqsdiJ"]').click()

        # 검색창에 커서 놓기
        search_part = driver.find_element(By.CSS_SELECTOR, 'input[class="sc-84bdfbae-4 ibdzoI"]')

        # 소분류 입력하기
        search_part.send_keys(f'{sub}')

        # enter 입력하기
        search_part.send_keys(Keys.RETURN)
        
        # enter 입력 후 로드가 충분히 될 수 있도록 기다린다.
        time.sleep(5)


        # 첫 화면에 들어갔다.

        # 정보 추출하는 것을 스코롤하면서 총 10번 추출한다.

        
        # 스크롤 10번 내리고
        for i in range(10) :
            
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            print(f'{main}_{sub}_{i+1}번 스크롤 다운 진행중...')
            time.sleep(3)
        
        print('곧 크롤링 시작')
        time.sleep(10)

        
            
        # 큰 박스를 먼저 찾는다. -> 가장 우선 순위는 이미지 url이 된다.
        large_boxes = driver.find_elements(By.CSS_SELECTOR, 'div.sc-2e746fd3-0.loRRgN')

        

        # 하나의 개별 품목에 대한 정보에 접근하기 위한 준비를 한다.
        for large_box in tqdm(large_boxes) :
            

            # [일단 필요한 정보는 다 찾는다.]

            # 큰 박스 > 작은 박스 1 찾기
            small_box_1 = large_box.find_element(By.CSS_SELECTOR, 'div.sc-2e746fd3-1.gHhzqc')

            # 작은 박스 1 > connect_url 찾기
            connect_url_element = small_box_1.find_element(By.CSS_SELECTOR, 'a[href]')
            connect_url = connect_url_element.get_attribute('href')

            # connect_url > div > img 태그 찾기
            img_tag = connect_url_element.find_element(By.CSS_SELECTOR, 'div > img.sc-b34a2ec2-0.bOxxOH')
            img_url = img_tag.get_attribute('src')

            # 큰 박스 > 작은 박스 2 찾기
            small_box_2 = large_box.find_element(By.CSS_SELECTOR, 'div.sc-2e746fd3-6.cVhNfy')

            # 작은 박스 2에서 가격 찾기
            price_element = small_box_2.find_element(By.CSS_SELECTOR, 'div.sc-2e746fd3-5.laAErS')
            price = price_element.text


            # 작은 박스 2에서 제목 찾기
            title_element = small_box_2.find_element(By.CSS_SELECTOR, 'div.sc-2e746fd3-5.RmlFc')
            title = title_element.text

            # 작은 박스 2에서 사이즈 찾기

            # 사이즈가 있으면
            try :
                size_element = small_box_2.find_element(By.CSS_SELECTOR, 'div[class="sc-2e746fd3-8 fnhBvW"]')
                size = size_element.text
            # 사이즈가 없으면
            except :
                size = 'Free'



            # [필요한 정보는 다 찾았으니 이제 해당 정보를 각 리스트에 넣어줄지 판단한다.]

            # 첫 시도, 백지 상태일 때 -> 1. img_url에 empty가 있으면 안된다. 2. img_urls에 img_url이 있으면 안된다.
            if ('empty' not in img_url) and (img_url not in img_urls) :     
                connect_urls.append(connect_url) 
                prices.append(price) 
                img_urls.append(img_url) 
                titles.append(title) 
                sizes.append(size)
                
    
        # 반복문을 지나면서 정보가 정리되었으니, 데이터프레임을 만들어준다.
        datas = pd.DataFrame({'price' : prices, 'img_url' : img_urls, 'connect_url' : connect_urls, 'title' :titles })

        datas.to_csv(f'./secondwear_crawling_results/{main}_{sub}.csv', index=False)
        print(f'{main}_{sub} 완료 : 탐색 제품 수 -> {len(img_urls)} ')       
                    
                


상의_니트_1번 스크롤 다운 진행중...
상의_니트_2번 스크롤 다운 진행중...
상의_니트_3번 스크롤 다운 진행중...
상의_니트_4번 스크롤 다운 진행중...
상의_니트_5번 스크롤 다운 진행중...
상의_니트_6번 스크롤 다운 진행중...
상의_니트_7번 스크롤 다운 진행중...
상의_니트_8번 스크롤 다운 진행중...
상의_니트_9번 스크롤 다운 진행중...
상의_니트_10번 스크롤 다운 진행중...
곧 크롤링 시작


100%|██████████| 330/330 [00:13<00:00, 24.77it/s]


상의_니트 완료 : 탐색 제품 수 -> 157 
상의_반소매 티셔츠_1번 스크롤 다운 진행중...
상의_반소매 티셔츠_2번 스크롤 다운 진행중...
상의_반소매 티셔츠_3번 스크롤 다운 진행중...
상의_반소매 티셔츠_4번 스크롤 다운 진행중...
상의_반소매 티셔츠_5번 스크롤 다운 진행중...
상의_반소매 티셔츠_6번 스크롤 다운 진행중...
상의_반소매 티셔츠_7번 스크롤 다운 진행중...
상의_반소매 티셔츠_8번 스크롤 다운 진행중...
상의_반소매 티셔츠_9번 스크롤 다운 진행중...
상의_반소매 티셔츠_10번 스크롤 다운 진행중...
곧 크롤링 시작


100%|██████████| 270/270 [00:11<00:00, 24.18it/s]


상의_반소매 티셔츠 완료 : 탐색 제품 수 -> 168 
상의_카라 티셔츠_1번 스크롤 다운 진행중...
상의_카라 티셔츠_2번 스크롤 다운 진행중...
상의_카라 티셔츠_3번 스크롤 다운 진행중...
상의_카라 티셔츠_4번 스크롤 다운 진행중...
상의_카라 티셔츠_5번 스크롤 다운 진행중...
상의_카라 티셔츠_6번 스크롤 다운 진행중...
상의_카라 티셔츠_7번 스크롤 다운 진행중...
상의_카라 티셔츠_8번 스크롤 다운 진행중...
상의_카라 티셔츠_9번 스크롤 다운 진행중...
상의_카라 티셔츠_10번 스크롤 다운 진행중...
곧 크롤링 시작


100%|██████████| 180/180 [00:07<00:00, 23.98it/s]


상의_카라 티셔츠 완료 : 탐색 제품 수 -> 73 
상의_베스트_1번 스크롤 다운 진행중...
상의_베스트_2번 스크롤 다운 진행중...
상의_베스트_3번 스크롤 다운 진행중...
상의_베스트_4번 스크롤 다운 진행중...
상의_베스트_5번 스크롤 다운 진행중...


KeyboardInterrupt: 